In [ ]:
# default_exp nbeats.init

In [ ]:
# hide
import sys

sys.path.append("..")
import pandas as pd
%load_ext autoreload
%autoreload 2

/home/tako/dev/env37/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/tako/dev/env37/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# N-Beats init

> A basic architecture for time series forecasting.


The approach is based on https://arxiv.org/abs/1905.10437


In [ ]:
# hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#export
from fastcore.utils import *
from fastcore.imports import *
from fastai.basics import *
from fastai.callback.hook import num_features_model
from fastai.callback.all import *
from fastai.torch_core import *
from torch.autograd import Variable
from fastseq.all import *
from fastseq.nbeats.model import *
from fastseq.nbeats.metrics import *
from fastseq.nbeats.learner import *

In [ ]:
horizon, lookback = 7,15
items = L(np.arange(-5,100)[None,:],np.arange(500,550)[None,:],np.arange(-110,-56)[None,:]).map(tensor)
data = TSDataLoaders.from_items(items, horizon = horizon, lookback=lookback, step=1, after_batch = NormalizeTS()
                               )

mdl = NBeatsNet(device = data.train.device, stack_types=('trend','seasonality'), horizon=horizon, lookback=lookback)
loss_func = F.mse_loss
learn = Learner(data, mdl, loss_func=loss_func, opt_func= Adam, 
                cbs=L(NBeatsTrainer())
               )

Train:74; Valid: 33; Test 3


In [ ]:
xb = data.one_batch()[0]

In [ ]:
def example_forward_hook(m,i,o): 
    print( i[0][0].shape, o[0][0].shape)
hook = mdl.register_forward_hook(example_forward_hook)
y = mdl(xb)
hook.remove()

torch.Size([1, 15]) torch.Size([1, 7])


In [ ]:
class Block(Module):
    def __init__(self, ):
        self.layers = nn.Sequential(LinBnDrop(2, 3),LinBnDrop(3,4))        
        self.att = LinBnDrop(4, 1)
    def forward(self, x):
        return self.att(self.layers(x))
    
mdl = Block()
print(mdl)

In [ ]:
print(sum([o for o in mdl.children()], []))

[Sequential(
  (0): LinBnDrop(
    (0): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=2, out_features=3, bias=False)
  )
  (1): LinBnDrop(
    (0): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=3, out_features=4, bias=False)
  )
), LinBnDrop(
  (0): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Linear(in_features=4, out_features=1, bias=False)
)]


In [ ]:
lin_layers = (nn.Conv1d, nn.Conv2d, nn.Conv3d, nn.Linear, nn.ReLU)
def find_modules(m, cond):
    print(m)
    print('-')
    if cond(m): 
        print(m._get_name())
        return [(m,)]
    return sum([find_modules(o,cond) for o in m.children()], [])
mods = find_modules(mdl, lambda o: isinstance(o,lin_layers))
mods

Block(
  (layers): Sequential(
    (0): LinBnDrop(
      (0): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=2, out_features=3, bias=False)
    )
    (1): LinBnDrop(
      (0): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=3, out_features=4, bias=False)
    )
  )
  (att): LinBnDrop(
    (0): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=4, out_features=1, bias=False)
  )
)
-
Sequential(
  (0): LinBnDrop(
    (0): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=2, out_features=3, bias=False)
  )
  (1): LinBnDrop(
    (0): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=3, out_features=4, bias=False)
  )
)
-
LinBnDrop(
  (0): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_runni

[(Linear(in_features=2, out_features=3, bias=False),),
 (Linear(in_features=3, out_features=4, bias=False),),
 (Linear(in_features=4, out_features=1, bias=False),)]

In [ ]:
def append_stat(hook, mod, outp):
    print(hook,mod,outp)
    assert False
    d = outp.data
    hook.mean,hook.std = d.mean().item(),d.std().item()
    
with Hooks(mods, append_stat) as hooks:
    mdl(xb)
    for hook in hooks:
        print(hook.mean,hook.std)

TypeError: append_stat() missing 1 required positional argument: 'outp'

In [ ]:
def lsuv_module(m, xb):
    h = Hook(m, append_stat)

    while mdl(xb) is not None and abs(h.mean)  > 1e-3: m.bias -= h.mean
    while mdl(xb) is not None and abs(h.std-1) > 1e-3: m.weight.data /= h.std

    h.remove()
    return h.mean,h.std

for m in mods: 
    print(lsuv_module(m, xb))

TypeError: append_stat() missing 1 required positional argument: 'outp'